In [ ]:
!pip install -U langchain transformers

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import warnings

In [ ]:
pdf_path = "/content/financials/audit_report.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [ ]:

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
docs = splitter.split_documents(documents)

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)
vectorstore.save_local("faiss_index")

In [ ]:
model_id = "TheBloke/zephyr-7B-beta-GGUF"
gguf_model_path = "/content/zephyr-7b.Q4_K_M.gguf"

model = AutoModelForCausalLM.from_pretrained(
    gguf_model_id,
    trust_remote_code=True,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(gguf_model_id)

In [ ]:
pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.1
)
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=False,
)

In [ ]:
def ask_audit_question(query):
    result = qa_chain(query)
    print("🔍 Answer:", result['result'])
    print("\n📎 Supporting Sources:\n")
    for doc in result['source_documents']:
        print("•", doc.page_content[:300], "\n---\n")

ask_audit_question("Are there any financial inconsistencies in the report?")